In [12]:
import pandas as pd
import webbrowser
import os
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, CustomJS, RangeSlider
from bokeh.layouts import column
from bokeh.palettes import Category20
from bokeh.transform import factor_cmap
from bokeh.io import output_file
from bokeh.resources import CDN
from bokeh.embed import file_html

# Load the data
df = pd.read_csv(r'C:\Users\jrjsw\Downloads\region_12.csv')

# Convert year to datetime for easier filtering
df['year'] = pd.to_datetime(df['iyear'], format='%Y')

# Replace NaN values with 0 for casualties
df['nkill'] = df['nkill'].fillna(0)
df['nwound'] = df['nwound'].fillna(0)

# Calculate total casualties
df['total_casualties'] = df['nkill'] + df['nwound']

# Get unique target types
target_types = sorted(df['targtype1_txt'].unique().tolist())

# Create color map
num_colors = min(len(target_types), 20)  # Category20 has a maximum of 20 colors
color_palette = Category20[num_colors]
color_map = factor_cmap('targtype1_txt', palette=color_palette, factors=target_types)

# Create ColumnDataSource
source = ColumnDataSource(df)

# Create the figure
p = figure(title="Target Types and Casualties", x_axis_label="Number Killed", y_axis_label="Number Wounded",
           width=1000, height=600, tools="pan,wheel_zoom,box_zoom,reset")

# Add circle glyphs
circles = p.circle(x='nkill', y='nwound', size='total_casualties', 
                   color=color_map, alpha=0.6, source=source, legend_field='targtype1_txt')

# Add hover tool
hover = HoverTool(tooltips=[
    ("Target", "@targtype1_txt"),
    ("Killed", "@nkill"),
    ("Wounded", "@nwound")
])
p.add_tools(hover)

# Customize legend
p.legend.click_policy = "hide"
p.legend.location = "right"
p.legend.label_text_font_size = "8pt"
p.legend.spacing = 1
p.legend.glyph_height = 15
p.legend.label_height = 15
p.legend.label_width = 150

# Create RangeSlider for year filtering
year_slider = RangeSlider(start=df['iyear'].min(), end=df['iyear'].max(),
                          value=(df['iyear'].min(), df['iyear'].max()),
                          step=1, title="Year Range")

# Create a JavaScript callback for the slider
callback = CustomJS(args=dict(source=source, original=source.data), code="""
    var data = source.data;
    var f = cb_obj.value;
    var original = original;
    
    for (var key in original) {
        data[key] = [];
    }
    
    for (var i = 0; i < original.iyear.length; i++) {
        if (original.iyear[i] >= f[0] && original.iyear[i] <= f[1]) {
            for (var key in original) {
                data[key].push(original[key][i]);
            }
        }
    }
    source.change.emit();
""")

year_slider.js_on_change('value', callback)

# Layout
layout = column(year_slider, p)

# Generate the HTML
html = file_html(layout, CDN, "Target Types and Casualties")

# Save the HTML to a file
file_path = os.path.join(os.getcwd(), "target_types_casualties.html")
with open(file_path, "w") as f:
    f.write(html)

# Open the HTML file using the default program
webbrowser.open('file://' + os.path.realpath(file_path))

print(f"Plot has been saved and opened as '{file_path}'.")

Plot has been saved and opened as 'C:\Users\jrjsw\target_types_casualties.html'.
